In [1]:
import os, json, pathlib
import boto3
from botocore.exceptions import ClientError

# Caminho para o arquivo de credenciais
creds_path = pathlib.Path("../secret/credentials.json").resolve()
if not creds_path.exists():
    raise FileNotFoundError(f"Não encontrei o arquivo de credenciais: {creds_path}.\nColoque seu credentials.json em 'secret/'.")

with open(creds_path, "r", encoding="utf-8") as f:
    creds = json.load(f)

required = ["aws_access_key_id", "aws_secret_access_key", "bucket"]
for k in required:
    if k not in creds:
        raise KeyError(f"Chave obrigatória ausente no credentials.json: '{k}'")

aws_access_key_id     = creds["aws_access_key_id"]
aws_secret_access_key = creds["aws_secret_access_key"]
aws_region            = creds.get("region", "sa-east-1")
bucket_name           = creds["bucket"]

# Cliente S3 (usa credenciais diretamente, sem gravar nada em disco)
s3 = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region,
)

print("Credenciais OK | Bucket:", bucket_name, "| Região:", aws_region)


Credenciais OK | Bucket: bucket-t2-avalensurance | Região: sa-east-1


In [2]:
# Listar alguns objetos 
def listar_objetos(bucket, prefix=None, max_items=50):
    paginator = s3.get_paginator('list_objects_v2')
    params = {"Bucket": bucket}
    if prefix:
        params["Prefix"] = prefix
    encontrados = []
    try:
        for page in paginator.paginate(**params):
            for obj in page.get("Contents", []):
                encontrados.append(obj["Key"])
                if len(encontrados) >= max_items:
                    return encontrados
    except ClientError as e:
        print("Erro ao listar objetos:", e)
    return encontrados

amostra = listar_objetos(bucket_name, max_items=30)
print(f"Primeiros {len(amostra)} objetos do bucket:")
for k in amostra:
    print(" -", k if len(k) < 120 else k[:117] + "...")


Primeiros 6 objetos do bucket:
 - clinical.csv
 - costs.csv
 - demographics.csv
 - lifestyle.csv
 - policy.csv
 - utilization.csv


In [3]:
# Função para baixar tudo 
import os
from pathlib import Path

def baixar_bucket(bucket, destino="../data", prefix=None):
    destino_path = Path(destino)
    destino_path.mkdir(parents=True, exist_ok=True)

    paginator = s3.get_paginator('list_objects_v2')
    params = {"Bucket": bucket}
    if prefix:
        params["Prefix"] = prefix

    total = 0
    try:
        for page in paginator.paginate(**params):
            for obj in page.get('Contents', []):
                key = obj['Key']
                # Mantém a estrutura de pastas local
                local_path = destino_path / key if not prefix else destino_path / key[len(prefix):].lstrip("/")
                local_path.parent.mkdir(parents=True, exist_ok=True)
                print("Baixando:", key)
                s3.download_file(bucket, key, str(local_path))
                total += 1
    except ClientError as e:
        print("Erro ao baixar:", e)
        raise

    print(f"Concluído: {total} arquivos baixados em: {destino_path.resolve()}")

# === EXECUÇÃO PADRÃO: baixar tudo para ../data ===
baixar_bucket(bucket_name, destino="../data")

# === EXEMPLO: baixar somente um prefixo específico ===
# baixar_bucket(bucket_name, destino="../data/recorte", prefix="pasta/no/s3/")


Baixando: clinical.csv
Baixando: costs.csv
Baixando: demographics.csv
Baixando: lifestyle.csv
Baixando: policy.csv
Baixando: utilization.csv
Concluído: 6 arquivos baixados em: C:\AmbienteProgrmacao\Projetos\avalensurance-bia\data


In [4]:
# Verificar quantos arquivos foram baixados e listar alguns exemplos
from pathlib import Path
base = Path("../data").resolve()
arquivos = [str(p.relative_to(base)) for p in base.rglob("*") if p.is_file()]
print(f"📦 {len(arquivos)} arquivos baixados.")
for k in arquivos[:20]:
    print(" -", k)


📦 6 arquivos baixados.
 - clinical.csv
 - costs.csv
 - demographics.csv
 - lifestyle.csv
 - policy.csv
 - utilization.csv


In [2]:
import pandas as pd
import numpy as np

# Caminho do CSV (saindo da pasta notebooks para a pasta data)
caminho = "../data/full_warehouse_merged.csv"

# Carregar CSV
df = pd.read_csv(caminho)

# Valores customizados que significam nulo
valores_nulos_custom = ["-1", -1, "no_data", "sem_registro"]

# Substituir esses valores por NaN
df = df.replace(valores_nulos_custom, np.nan)

# Calcular número de nulos por coluna
num_nulos = df.isnull().sum()

# Calcular porcentagem de nulos por coluna
pct_nulos = (df.isnull().mean() * 100).round(2)

# Criar DataFrame final
tabela_nulos = pd.DataFrame({
    "num_nulos": num_nulos,
    "pct_nulos": pct_nulos
})

print("\n===== TABELA DE NULOS =====\n")
print(tabela_nulos)



===== TABELA DE NULOS =====

                             num_nulos  pct_nulos
person_id                            0       0.00
annual_medical_cost               6380       9.85
annual_premium                       0       0.00
monthly_premium                      0       0.00
claims_count                         0       0.00
avg_claim_amount                     0       0.00
total_claims_paid                    0       0.00
hypertension                         0       0.00
diabetes                             0       0.00
asthma                               0       0.00
copd                                 0       0.00
cardiovascular_disease               0       0.00
cancer_history                       0       0.00
kidney_disease                       0       0.00
liver_disease                        0       0.00
arthritis                            0       0.00
mental_health                        0       0.00
chronic_count                        0       0.00
systolic_bp         